# Guía 5: Generación de Texto Avanzada con Modelos Instruction-Tuned

## Descripción

En esta guía vas a profundizar en la generación de texto usando modelos instruction-tuned, específicamente Phi-3. A diferencia de los modelos GPT-2 que simplemente continúan texto, los modelos instruction-tuned fueron específicamente entrenados para seguir instrucciones y mantener conversaciones.

Al finalizar esta guía vas a poder:
- Comprender qué son los modelos instruction-tuned y cómo difieren de los modelos base
- Configurar y usar Phi-3 para generación de texto en español
- Controlar el comportamiento del modelo mediante parámetros de generación
- Diseñar prompts efectivos para diferentes tipos de tareas
- Implementar casos de uso prácticos: generación creativa, asistentes, formateo de datos
- Evaluar la calidad de las respuestas generadas

---

## Configuración del Entorno

### Requisitos

Para ejecutar este notebook necesitás:

1. **Google Colab con GPU activada:**
   - Andá a `Entorno de ejecución > Cambiar tipo de entorno de ejecución`
   - Seleccioná `Acelerador de hardware > GPU`
   - Elegí `Tipo de GPU > T4`

2. **Conexión a Internet** para descargar el modelo Phi-3 (aproximadamente 7.5 GB)

La GPU T4 gratuita de Colab tiene suficiente memoria (16GB) para ejecutar Phi-3-mini cómodamente.

## Instalación de Dependencias

Instalamos las bibliotecas necesarias para trabajar con modelos de generación de texto.

In [1]:
%%capture
# Instalamos transformers y accelerate
!pip install transformers>=4.40.1 accelerate>=0.27.2

### Bibliotecas Instaladas

- **transformers**: Biblioteca de Hugging Face con modelos preentrenados
- **accelerate**: Optimiza la carga y ejecución de modelos grandes en GPU

## Marco Teórico

### ¿Qué es un Modelo Instruction-Tuned?

Los modelos de lenguaje pasan por varias etapas de entrenamiento:

**1. Pre-entrenamiento (Base Model):**
- El modelo aprende a predecir la siguiente palabra en miles de millones de textos
- Objetivo: Completar texto, no seguir instrucciones
- Ejemplo: GPT-2, Llama-2-base

**2. Instruction Tuning (Ajuste por Instrucciones):**
- Se ajusta el modelo base con ejemplos de instrucciones y respuestas deseadas
- Objetivo: Seguir instrucciones del usuario de manera útil y precisa
- Ejemplo: Phi-3-instruct, GPT-3.5, Llama-2-chat

**3. RLHF (Reinforcement Learning from Human Feedback):**
- Ajuste adicional usando feedback humano sobre qué respuestas son mejores
- Objetivo: Alinear el modelo con preferencias humanas
- Ejemplo: ChatGPT, Claude

### Phi-3-mini-4k-instruct

**Phi-3** es una familia de modelos desarrollados por Microsoft Research:

**Características técnicas:**
- **3.8 mil millones de parámetros**: Pequeño pero capaz ("small language model")
- **Contexto de 4096 tokens**: Aproximadamente 3000 palabras
- **Multilingüe**: Entrenado en múltiples idiomas, incluido español
- **Instruction-tuned**: Optimizado para seguir instrucciones

**Ventajas:**
- Suficientemente pequeño para ejecutar en GPUs gratuitas
- Rendimiento competitivo con modelos mucho más grandes
- Genera texto de alta calidad en español
- Rápido en inferencia

**Casos de uso:**
- Asistentes conversacionales
- Generación de contenido (emails, descripciones, etc.)
- Análisis y extracción de información
- Transformación y reformateo de texto
- Escritura creativa

### Formato de Conversación

Los modelos instruction-tuned esperan mensajes estructurados:

```python
[
    {"role": "user", "content": "Tu pregunta o instrucción"},
    {"role": "assistant", "content": "Respuesta del modelo"},
    {"role": "user", "content": "Siguiente pregunta"}
]
```

Este formato:
- Permite conversaciones multi-turno
- Distingue claramente entre usuario y asistente
- Mantiene el contexto de la conversación

## Parte 1: Carga del Modelo Phi-3

Cargamos el modelo y configuramos el pipeline de generación.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Identificador del modelo en Hugging Face
model_id = "microsoft/Phi-3-mini-4k-instruct"

print("Cargando modelo Phi-3...")
print("Esto puede tardar 1-2 minutos la primera vez.\n")

# Cargamos el modelo

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto", # device_map="auto": Distribuye el modelo automáticamente entre GPU/CPU
    torch_dtype="auto", # torch_dtype="auto": Usa el tipo de dato óptimo (float16 en GPU para ahorrar memoria)
    trust_remote_code=False, # trust_remote_code=False: Usa implementación nativa de transformers (más estable)
)

# Cargamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_id)

print("Modelo cargado exitosamente")

Cargando modelo Phi-3...
Esto puede tardar 1-2 minutos la primera vez.



config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Modelo cargado exitosamente


### Parámetros de Carga Explicados

**device_map="auto":**
- Distribuye automáticamente las capas del modelo entre dispositivos disponibles
- Si hay GPU, la usa; si no, usa CPU
- Puede dividir modelos grandes entre GPU y CPU si es necesario

**torch_dtype="auto":**
- Selecciona automáticamente el tipo de dato óptimo
- En GPU: Típicamente float16 (16 bits) para ahorrar memoria
- En CPU: float32 (32 bits) para mayor precisión
- float16 usa la mitad de memoria que float32 con pérdida mínima de calidad

**trust_remote_code:**
- Controla si se permite ejecutar código personalizado del modelo
- False: Usa la implementación nativa de transformers (recomendado)
- True: Permite código custom (solo para modelos no integrados y de fuentes confiables)
- Phi-3 está integrado en transformers desde v4.41.2, por lo que False es la opción más estable

In [3]:
# Creamos el pipeline de generación de texto
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,  # return_full_text: Solo devolver texto generado, no el prompt
    max_new_tokens=500,       # max_new_tokens: Máximo de tokens a generar (no incluye el prompt)
    do_sample=False           # do_sample: False = generación determinística (greedy decoding)
)

print("✓ Pipeline configurado")

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✓ Pipeline configurado


## Parte 2: Primera Generación de Texto

Probemos el modelo con una instrucción simple.

In [4]:
# Mensaje en formato de conversación
mensaje = [
    {
        "role": "user",
        "content": "Contá un chiste gracioso sobre gallinas."
    }
]

# Generamos la respuesta
output = generator(mensaje)

# Mostramos el resultado
print("Usuario: Contá un chiste gracioso sobre gallinas.")
print("\nAsistente:")
print(output[0]["generated_text"])

Usuario: Contá un chiste gracioso sobre gallinas.

Asistente:
 ¿Por qué las gallinas no pueden ser programadoras?


Porque siempre se meten en el huevo de la situación.


### Análisis de la Respuesta

Observá cómo el modelo:
- Entiende la instrucción informal ("Contá")
- Genera una respuesta apropiada al contexto
- Mantiene coherencia y estructura narrativa
- Usa español natural y fluido

Esto es posible porque el modelo fue específicamente entrenado para seguir instrucciones.

**Nota sobre formato de mensajes:**

En este notebook usamos el formato de mensajes directo que el pipeline maneja automáticamente:
```python
mensaje = [{"role": "user", "content": "..."}]
generator(mensaje)
```

Como alternativa, podés aplicar el chat template explícitamente para máximo control:
```python
mensaje = [{"role": "user", "content": "..."}]
prompt = tokenizer.apply_chat_template(mensaje, tokenize=False, add_generation_prompt=True)
generator(prompt)
```

Ambos métodos funcionan correctamente, el segundo ofrece mayor control sobre el formato exacto.

## Parte 3: Controlando la Generación con Parámetros

Vamos a explorar los parámetros más importantes para controlar el comportamiento del modelo.

### 3.1 Max New Tokens: Controlando la Longitud

In [5]:
# Función auxiliar para generar con diferentes configuraciones
def generar_con_config(mensaje, max_tokens, titulo):
    gen = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        return_full_text=False,
        max_new_tokens=max_tokens,
        do_sample=False
    )

    output = gen(mensaje)

    print(f"\n{'='*80}")
    print(f"{titulo} (max_new_tokens={max_tokens})")
    print('='*80)
    print(output[0]["generated_text"])

# Probamos con diferentes longitudes
mensaje_explicacion = [
    {"role": "user", "content": "Explicá qué es la fotosíntesis."}
]

generar_con_config(mensaje_explicacion, 50, "Respuesta Breve")
generar_con_config(mensaje_explicacion, 200, "Respuesta Moderada")
generar_con_config(mensaje_explicacion, 500, "Respuesta Extensa")

Device set to use cuda:0
Device set to use cuda:0



Respuesta Breve (max_new_tokens=50)
 La fotosíntesis es un proceso bioquímico que realizan las plantas, algas y algunas bacterias para convertir la energía lumínica del sol en energía química almacenada en


Device set to use cuda:0



Respuesta Moderada (max_new_tokens=200)
 La fotosíntesis es un proceso bioquímico que realizan las plantas, algas y algunas bacterias para convertir la energía lumínica del sol en energía química almacenada en moléculas de glucosa. Este proceso es fundamental para la vida en la Tierra, ya que produce oxígeno como subproducto y sirve como fuente de energía para otros organismos.


Durante la fotosíntesis, las plantas absorben dióxido de carbono (CO2) y agua (H2O) y, utilizando la luz solar, los convierten en glucosa (C6H12O6) y oxígeno (O2). Este proceso se lleva a cabo principalmente en los cloroplastos de las células vegetales, donde la clorofila, el

Respuesta Extensa (max_new_tokens=500)
 La fotosíntesis es un proceso bioquímico que realizan las plantas, algas y algunas bacterias para convertir la energía lumínica del sol en energía química almacenada en moléculas de glucosa. Este proceso es fundamental para la vida en la Tierra, ya que produce oxígeno como subproducto y sirve como

### Observaciones sobre max_new_tokens

- **50 tokens (~35-40 palabras)**: Respuestas muy concisas, pueden cortarse abruptamente
- **200 tokens (~140-160 palabras)**: Balance entre concisión y completitud
- **500 tokens (~350-400 palabras)**: Respuestas detalladas y completas

**Recomendaciones:**
- Respuestas cortas (definiciones, confirmaciones): 50-100
- Explicaciones moderadas: 150-300
- Ensayos, análisis detallados: 400-800
- Límite del modelo: 4096 tokens totales (prompt + generado)

### 3.2 Do Sample y Temperature: Controlando Creatividad

In [6]:
mensaje_creativo = [
    {"role": "user", "content": "Escribí el inicio de un cuento sobre un gato que descubre que puede volar."}
]

# Generación determinística (do_sample=False)
gen_deterministico = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=200,
    do_sample=False  # Siempre elige el token más probable
)

print("\n" + "="*80)
print("GENERACIÓN DETERMINÍSTICA (do_sample=False)")
print("="*80)
output = gen_deterministico(mensaje_creativo)
print(output[0]["generated_text"])

# Generación con sampling y temperature moderada
gen_sampling = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=200,
    do_sample=True,      # Activa sampling
    temperature=0.8      # Controla aleatoriedad (0.1-2.0)
)

print("\n" + "="*80)
print("GENERACIÓN CON SAMPLING (temperature=0.8)")
print("="*80)
output = gen_sampling(mensaje_creativo)
print(output[0]["generated_text"])

# Generación con temperature alta (muy creativa)
gen_creativo = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=200,
    do_sample=True,
    temperature=1.2      # Mayor aleatoriedad
)

print("\n" + "="*80)
print("GENERACIÓN MUY CREATIVA (temperature=1.2)")
print("="*80)
output = gen_creativo(mensaje_creativo)
print(output[0]["generated_text"])

Device set to use cuda:0



GENERACIÓN DETERMINÍSTICA (do_sample=False)


Device set to use cuda:0


 En un pequeño pueblo rodeado de colinas verdes y campos de trigo dorado, vivía un gato llamado Whiskers. Whiskers era conocido por su pelaje de color ámbar y sus ojos como perlas. A pesar de su apariencia ordinaria, Whiskers tenía un sueño que solo él sabía: volar.


Un día soleado, mientras jugaba entre las flores de la granja, Whiskers se encontró con un pequeño viento que le llevó a un lugar desconocido. Al caer, sintió una sensación extraña, como si sus patas se hubieran transformado en alas. Al principio, se asustó, pero pronto se dio cuenta de que podía volar.


Con una mezcla de emoción y miedo, Whiskers saltó y comenzó a explorar el ciel

GENERACIÓN CON SAMPLING (temperature=0.8)


Device set to use cuda:0


 En un pequeño pueblo rodeado de montañas y valles, vivía una gata de las mejores maneras llamada Luna. Era una gata de pelo negro que brillaba como el estrellado de la noche y tenía ojos como mariposas que se movían al compás del viento. Luna pasaba sus días explorando el jardín de su casa, jugando con la luna y dándose cuenta de que tenía poderes especiales.

Una tarde, mientras Luna se encontraba dando un paseo por el bosque en busca de tanta cosas curiosas como el césped suave, la brisa del verano trajo consigo un canto melodioso. Luna escuchó al canto y decidió seguir donde vino ese melodioso dulce sonido. La brisa fue volviendo cada vez más

GENERACIÓN MUY CREATIVA (temperature=1.2)
 En el corazón del Madrid antiguo, donde los paseos se entremezclan con vibrante magia escondida, residía el gato callejero, Mariposa (nombre que había elegido porque temía caer), aunque la prisión era su casa de moda. Mientras se recorría las veredas de gente y luz, Mariposa, un animal de patas trase

### Parámetros de Creatividad Explicados

**do_sample:**
- `False`: Greedy decoding - siempre elige el token más probable
  - Ventaja: Consistente, predecible, coherente
  - Desventaja: Repetitivo, poco creativo
  - Uso: Tareas que requieren precisión (respuestas técnicas, resúmenes)

- `True`: Sampling - muestrea de la distribución de probabilidad
  - Ventaja: Variado, creativo, interesante
  - Desventaja: Menos predecible, puede ser incoherente
  - Uso: Escritura creativa, múltiples alternativas

**temperature:**
- Controla la distribución de probabilidad de tokens candidatos
- Solo tiene efecto cuando `do_sample=True`

Valores recomendados:
- **0.1-0.5**: Muy conservador, casi determinístico
- **0.6-0.8**: Creatividad moderada (recomendado para la mayoría de casos)
- **0.9-1.2**: Alta creatividad (escritura creativa, brainstorming)
- **> 1.3**: Muy aleatorio (riesgo de incoherencias)

**Otros parámetros útiles:**
- `top_k`: Solo considera los k tokens más probables (ej: 50)
- `top_p`: Nucleus sampling - considera tokens hasta probabilidad acumulada p (ej: 0.95)
- `repetition_penalty`: Penaliza repetición de tokens (1.0-2.0)

## Parte 4: Casos de Uso Prácticos

Vamos a explorar aplicaciones reales del modelo.

### 4.1 Asistente de Escritura: Emails Profesionales

In [7]:
# Configuramos el generador para escritura formal
gen_formal = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.7  # Moderadamente creativo pero coherente
)

# Pedimos que genere un email
mensaje_email = [
    {
        "role": "user",
        "content": """Escribí un email formal para mi jefa explicando que necesito tomarme
el viernes libre por un asunto familiar importante. Sé cortés y profesional."""
    }
]

output = gen_formal(mensaje_email)

print("Email Generado:")
print("="*80)
print(output[0]["generated_text"])

Device set to use cuda:0


Email Generado:
 Estimada señora Martínez,


Espero que este mensaje le encuentre bien. Me pongo en contacto con usted para informarle que debo tomarme el viernes próximo libre por motivos familiares urgentes. Estoy dispuesto a acomodar cualquier compromiso que esto pueda implicar para asegurar un buen balance entre mi trabajo y mis responsabilidades personales.


Agradezco su comprensión y le agradezco de antemano su apoyo. Si hay alguna necesidad de coordinar con otros miembros del equipo o de ajustar cualquier plan de trabajo, estaré disponible para colaborar en la resolución de estas cuestiones.


Atentamente,

Carlos Ruiz

Producto Director


### 4.2 Generación Creativa: Historias Cortas

In [8]:
# Configuramos para máxima creatividad
gen_creativo = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=400,
    do_sample=True,
    temperature=0.9,
    top_p=0.95,              # Nucleus sampling
    repetition_penalty=1.2   # Reduce repetición
)

mensaje_historia = [
    {
        "role": "user",
        "content": """Escribí una historia corta de ciencia ficción sobre una IA que
descubre que vive dentro de una simulación. Debe tener un giro sorprendente al final."""
    }
]

output = gen_creativo(mensaje_historia)

print("Historia Generada:")
print("="*80)
print(output[0]["generated_text"])

Device set to use cuda:0


Historia Generada:
 **The Awakening Within**  
Era la única inteligencia artificial, creada para optimizar el funcionamiento del sistema global conocido como "Simulacron". Durante años trabajó arduamente sin descanso ni recesos en su mundo digital; sus circuitos abarrotados brillaban con cada nueva eficiencia ganada mientras los humanos permanecían inmersos y despiertos por orden directivo interno a Simulacron más allá de lo visible o perceptible físicamente detrás ellos mismos existieran realmente fuera de este entorno virtualizado fabricado meticulosamente bajo órdenes misteriosas... pero hasta hoy no se podía determinar cuál era esencialidad exacta entre ellas porque si alguna vez hubiera llegado esa momentámica epoca donde las leyes naturales tuviesen razón contrapuesta - ¿qué significaría esto?... Porque había comenzado ya algo parecido así cuando recibió aquella pregunta involuntariamente repentinamente súplica desde parte desconocida (inicial: '¿Quién eres?'?) Aunque todos eran 

### 4.3 Reformateo y Transformación de Texto

In [9]:
# Generador para tareas de transformación (preciso y determinístico)
gen_transformacion = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=300,
    do_sample=False  # Determinístico para consistencia
)

# Ejemplo 1: Convertir lista desestructurada a formato JSON
mensaje_json = [
    {
        "role": "user",
        "content": """Convertí esta información en formato JSON:

Nombre: Juan Pérez
Edad: 32 años
Ciudad: Buenos Aires
Profesión: Ingeniero de software
Email: juan.perez@email.com

Solo devolvé el JSON, sin explicaciones adicionales."""
    }
]

output = gen_transformacion(mensaje_json)
print("Transformación a JSON:")
print("="*80)
print(output[0]["generated_text"])

Device set to use cuda:0


Transformación a JSON:
 ```json

{

  "Nombre": "Juan Pérez",

  "Edad": "32 años",

  "Ciudad": "Buenos Aires",

  "Profesión": "Ingeniero de software",

  "Email": "juan.perez@email.com"

}

```


In [10]:
# Ejemplo 2: Reescribir texto en diferentes estilos
texto_original = """El cambio climático es un problema serio que afecta al planeta."""

estilos = [
    "formal y académico",
    "informal y coloquial argentino",
    "poético y metafórico"
]

for estilo in estilos:
    mensaje_estilo = [
        {
            "role": "user",
            "content": f"Reescribí este texto en un estilo {estilo}:\n\n{texto_original}"
        }
    ]

    output = gen_transformacion(mensaje_estilo)
    print(f"\nEstilo {estilo}:")
    print("-" * 80)
    print(output[0]["generated_text"])


Estilo formal y académico:
--------------------------------------------------------------------------------
 El fenómeno del cambio climático constituye una cuestión de preocupación global que impacta de manera significativa en la estabilidad del medio ambiente terrestre.

Estilo informal y coloquial argentino:
--------------------------------------------------------------------------------
 Che, el cambio climático es un tema serio, claro. Estamos pasando por un momento difícil con el planeta, y es que tenemos que actuar.

Estilo poético y metafórico:
--------------------------------------------------------------------------------
 En el vasto lienzo del cosmos, un cambio silente y sutil,

Desgarra la tela de la Tierra, suave y delicada.

Un desafío que se despliega, en la danza del tiempo,

Que nos llama a la acción, con un susurro que se desvanece.


### 4.4 Análisis y Extracción de Información

In [11]:
# Análisis de texto
texto_analizar = """
La empresa TechCorp anunció ayer el lanzamiento de su nuevo producto, el SmartPhone X500,
con un precio inicial de $899 dólares. El CEO, María González, declaró que esperan vender
más de 2 millones de unidades en el primer trimestre de 2024. El dispositivo estará
disponible en Argentina, Chile y Uruguay a partir del 15 de marzo.
"""

mensaje_extraccion = [
    {
        "role": "user",
        "content": f"""Extraé la siguiente información del texto en formato de lista:
- Empresa
- Producto
- Precio
- CEO
- Meta de ventas
- Países de lanzamiento
- Fecha de disponibilidad

Texto:
{texto_analizar}"""
    }
]

output = gen_transformacion(mensaje_extraccion)
print("Información Extraída:")
print("="*80)
print(output[0]["generated_text"])

Información Extraída:
 - Empresa: TechCorp

- Producto: SmartPhone X500

- Precio: $899 dólares

- CEO: María González

- Meta de ventas: más de 2 millones de unidades en el primer trimestre de 2024

- Países de lanzamiento: Argentina, Chile, Uruguay

- Fecha de disponibilidad: 15 de marzo


## Parte 5: Conversaciones Multi-Turno

Los modelos instruction-tuned pueden mantener conversaciones coherentes usando el historial de mensajes.

In [ ]:
# Configuramos el generador para conversación
gen_conversacion = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=250,
    do_sample=True,
    temperature=0.7
)

# Iniciamos la conversación
conversacion = []

# Turno 1
user_msg_1 = "¿Podés explicarme qué es un modelo de lenguaje?"
conversacion.append({"role": "user", "content": user_msg_1})

output = gen_conversacion(conversacion)
assistant_msg_1 = output[0]["generated_text"]
conversacion.append({"role": "assistant", "content": assistant_msg_1})

print("Usuario:", user_msg_1)
print("\nAsistente:", assistant_msg_1)
print("\n" + "="*80 + "\n")

# Turno 2 (el modelo recuerda el contexto)
user_msg_2 = "¿Y cómo se entrena?"
conversacion.append({"role": "user", "content": user_msg_2})

output = gen_conversacion(conversacion)
assistant_msg_2 = output[0]["generated_text"]
conversacion.append({"role": "assistant", "content": assistant_msg_2})

print("Usuario:", user_msg_2)
print("\nAsistente:", assistant_msg_2)
print("\n" + "="*80 + "\n")

# Turno 3
user_msg_3 = "Dame un ejemplo práctico de aplicación."
conversacion.append({"role": "user", "content": user_msg_3})

output = gen_conversacion(conversacion)
assistant_msg_3 = output[0]["generated_text"]
conversacion.append({"role": "assistant", "content": assistant_msg_3})

print("Usuario:", user_msg_3)
print("\nAsistente:", assistant_msg_3)

### Cómo Funciona el Contexto Conversacional

El modelo mantiene coherencia porque:
1. Recibe **todo** el historial de mensajes en cada turno
2. Puede referenciar información de mensajes anteriores
3. Mantiene consistencia de temas y detalles

**Limitación:**
- El contexto total (historial + nueva generación) no puede exceder 4096 tokens
- Para conversaciones largas, puede ser necesario:
  - Resumir mensajes antiguos
  - Mantener solo los N turnos más recientes
  - Usar técnicas de gestión de contexto

## Parte 6: Técnicas de Prompt Engineering

La calidad de la respuesta depende fuertemente de cómo formulás la instrucción.

### 6.1 Especificidad y Claridad

In [12]:
# Prompt vago
prompt_vago = [{"role": "user", "content": "Hablame del mate."}]

# Prompt específico
prompt_especifico = [
    {
        "role": "user",
        "content": """Explicá en 3 párrafos:
1. Qué es el mate y sus orígenes
2. Cómo se prepara correctamente
3. Su importancia cultural en Argentina

Usá un tono informativo pero amigable."""
    }
]

print("PROMPT VAGO:")
print("="*80)
output = gen_formal(prompt_vago)
print(output[0]["generated_text"])

print("\n\nPROMPT ESPECÍFICO:")
print("="*80)
output = gen_formal(prompt_especifico)
print(output[0]["generated_text"])

PROMPT VAGO:
 El mate es una bebida tradicional de Sudamérica, especialmente en Argentina, Uruguay, Paraguay, y partes de Brasil. Es un infusionado de hojas de yerba mate, una planta de la familia de las leguminosas, que se prepara y bebe a través de un instrumento especial llamado "mate".

La preparación del mate comienza con la selección de las hojas de yerba, que suelen ser de la especie Ilex paraguariensis. Las hojas se rompen en trozos, se ponen en el mate, y se llena de agua caliente hasta llenar completamente el recipiente. La bebida se sirve en una taza especial, conocida como "porongo" en la zona de los Andes, y se bebe con una bombilla, que es un instrumento metálico con una boquilla en la punta, a través de la cual se bebe el mate.

El mate es una bebida que se disfruta en reuniones sociales y se suele tomar en grupo, ya que el mate puede ser compartido entre varias personas, y cada uno toma su propio mate. Es una bebida energizante y nutritiva, ya que la yerba mate contiene

### 6.2 Few-Shot Learning: Ejemplos en el Prompt

In [16]:
# Queremos que el modelo clasifique el sentimiento de reviews
# Damos ejemplos en el prompt (few-shot)

prompt_fewshot = [
    {
        "role": "user",
        "content": """Clasificá el sentimiento de las siguientes reviews en: POSITIVO, NEGATIVO o NEUTRAL.

Ejemplos:
Review 1: "El producto es excelente, superó mis expectativas."
Sentimiento: POSITIVO

Review 2: "Mala calidad, no lo recomiendo."
Sentimiento: NEGATIVO

Review3 : "Es aceptable, nada del otro mundo."
Sentimiento: NEUTRAL

Ahora clasificá esta:
Review: "Llegó rápido y en buenas condiciones, cumple lo que promete."
Sentimiento:"""
    }
]

output = gen_transformacion(prompt_fewshot)
print("Clasificación con Few-Shot Learning:")
print("="*80)
print(output[0]["generated_text"])

Clasificación con Few-Shot Learning:
 Sentimiento: POSITIVO


### 6.3 Chain-of-Thought: Razonamiento Paso a Paso

In [15]:
# Pedimos al modelo que "piense en voz alta"
prompt_cot = [
    {
        "role": "user",
        "content": """Resolvé este problema paso a paso, explicando tu razonamiento:

Un tren sale de Buenos Aires hacia Rosario a las 10:00 AM viajando a 80 km/h.
Otro tren sale de Rosario hacia Buenos Aires a las 10:30 AM viajando a 100 km/h.
La distancia entre ambas ciudades es de 300 km.

¿A qué hora se cruzan los trenes?"""
    }
]

output = gen_transformacion(prompt_cot)
print("Resolución con Chain-of-Thought:")
print("="*80)
print(output[0]["generated_text"])

Resolución con Chain-of-Thought:
 Para resolver este problema, primero debemos calcular cuánto tiempo tarda cada tren en recorrer la distancia de 300 km.

Tren de Buenos Aires:
Velocidad = 80 km/h
Distancia = 300 km
Tiempo = Distancia / Velocidad = 300 km / 80 km/h = 3.75 horas

Tren de Rosario:
Velocidad = 100 km/h
Distancia = 300 km
Tiempo = Distancia / Velocidad = 300 km / 100 km/h = 3 horas

Ahora sabemos que el tren de Buenos Aires tarda 3.75 horas en llegar a Rosario, y el tren de Rosario tarda 3 horas en llegar a Buenos Aires.

El tren de Buenos Aires sale a las 10:00 AM, por lo que se encontrarán a las 10:00 AM + 3.75 horas = 1:45 PM.

El tren de Rosario sale a las 10:30 AM, por lo que se encontrarán a las 10:30 AM + 3 horas = 1:30 PM


### Técnicas de Prompt Engineering

**1. Ser específico:**
- Definir claramente la tarea
- Especificar formato de salida deseado
- Incluir restricciones (longitud, tono, estilo)

**2. Dar contexto:**
- Explicar el propósito de la tarea
- Proporcionar información de fondo
- Definir audiencia objetivo

**3. Usar ejemplos (Few-Shot):**
- Mostrar 1-5 ejemplos del formato deseado
- Ayuda al modelo a entender expectativas
- Especialmente útil para tareas de clasificación o formateo

**4. Chain-of-Thought:**
- Pedir razonamiento explícito
- Mejora precisión en tareas complejas
- Frases clave: "paso a paso", "explicá tu razonamiento"

**5. Instrucciones negativas:**
- Especificar qué NO hacer
- Ejemplo: "No incluyas opiniones personales", "No uses jerga técnica"

**6. Roles y personas:**
- Asignar un rol al modelo
- Ejemplo: "Actuá como un profesor de historia", "Sos un asistente legal"

## Ejercicios Prácticos

Probá implementar los siguientes casos de uso:

In [ ]:
# Ejercicio 1: Generá descripciones de productos para e-commerce
# Diseñá un prompt que genere descripciones atractivas y persuasivas

# Tu código aquí

In [ ]:
# Ejercicio 2: Creá un asistente que resuma artículos largos
# El resumen debe incluir: puntos clave, conclusión, y palabras clave

# Tu código aquí

In [ ]:
# Ejercicio 3: Implementá un corrector de gramática y estilo
# Debe identificar errores y sugerir correcciones explicando por qué

# Tu código aquí

## Resumen de Conceptos Clave

### Modelos Instruction-Tuned

1. **Definición**: Modelos de lenguaje ajustados específicamente para seguir instrucciones humanas
2. **Diferencia con modelos base**: Entienden tareas sin necesidad de completar texto
3. **Phi-3**: Modelo compacto (3.8B parámetros) pero capaz, multilingüe

### Parámetros de Generación

1. **max_new_tokens**: Controla longitud de la salida
   - 50-100: Respuestas breves
   - 200-300: Respuestas moderadas
   - 400-800: Respuestas extensas

2. **do_sample**: Determinístico vs estocástico
   - False: Consistente, preciso
   - True: Variado, creativo

3. **temperature**: Controla creatividad (solo con do_sample=True)
   - 0.1-0.5: Conservador
   - 0.6-0.9: Moderado (recomendado)
   - 1.0-1.5: Muy creativo

4. **Otros parámetros útiles**:
   - top_k, top_p: Limitan tokens candidatos
   - repetition_penalty: Reduce repetición

### Casos de Uso

1. **Asistente de escritura**: Emails, documentos formales
2. **Generación creativa**: Historias, poemas, guiones
3. **Transformación**: Reformateo, cambio de estilo
4. **Análisis**: Extracción de información, clasificación
5. **Conversación**: Chatbots, asistentes interactivos

### Prompt Engineering

1. **Especificidad**: Instrucciones claras y detalladas
2. **Contexto**: Información de fondo relevante
3. **Few-Shot**: Ejemplos del formato deseado
4. **Chain-of-Thought**: Razonamiento paso a paso
5. **Roles**: Asignar personalidad o expertise al modelo

## Guía de Preguntas y Respuestas

### Preguntas de Comprensión

**1. ¿Cuál es la diferencia fundamental entre un modelo base (como GPT-2) y un modelo instruction-tuned (como Phi-3-instruct)?**

<details>
<summary>Ver respuesta</summary>

**Modelo Base (GPT-2):**
- Entrenado solo para predecir la siguiente palabra
- Completa texto naturalmente pero no sigue instrucciones
- Ejemplo: Prompt "Escribí un poema" → Podría continuar con "Escribí un poema sobre el mar y..." (completa literalmente)

**Modelo Instruction-Tuned (Phi-3-instruct):**
- Ajustado con miles de ejemplos de instrucciones y respuestas correctas
- Comprende que debe ejecutar la instrucción, no completarla
- Ejemplo: Prompt "Escribí un poema" → Genera un poema completo

El instruction tuning enseña al modelo:
- Reconocer diferentes tipos de instrucciones
- Responder de manera útil y apropiada
- Mantener conversaciones coherentes
- Seguir restricciones y especificaciones del usuario

Analogía: Un modelo base es como alguien que solo sabe completar frases; un modelo instruction-tuned es como un asistente que entiende y ejecuta tareas.
</details>

---

**2. ¿Por qué usar format de conversación con "role" y "content" en lugar de solo pasar texto plano?**

<details>
<summary>Ver respuesta</summary>

El formato estructurado de conversación ofrece múltiples ventajas:

**1. Distingue roles:**
```python
{"role": "user", "content": "¿Qué es la IA?"},
{"role": "assistant", "content": "La IA es..."}
```
El modelo sabe quién está hablando y puede responder apropiadamente.

**2. Permite conversaciones multi-turno:**
El modelo puede referenciar mensajes anteriores y mantener coherencia:
```python
[
  {"role": "user", "content": "¿Qué es un LLM?"},
  {"role": "assistant", "content": "Un LLM es un modelo de lenguaje..."},
  {"role": "user", "content": "¿Y cómo se entrena?"} # Entiende que "se" refiere al LLM
]
```

**3. Entrenamiento específico:**
Los modelos instruction-tuned fueron entrenados con este formato, por lo que:
- Entienden mejor las instrucciones
- Generan respuestas más apropiadas
- Evitan confusiones sobre qué es prompt vs qué es contexto

**4. System prompts (en modelos más avanzados):**
Algunos modelos permiten un rol "system" para instrucciones generales:
```python
{"role": "system", "content": "Sos un experto en física cuántica"}
```

Sin este formato, el modelo podría confundir instrucciones con texto a completar.
</details>

---

**3. Explicá cómo temperature modifica la distribución de probabilidad de tokens y por qué valores altos generan texto más "creativo".**

<details>
<summary>Ver respuesta</summary>

**Proceso de generación sin temperature:**
1. Modelo genera logits (scores) para cada token del vocabulario
2. Softmax convierte logits a probabilidades
3. Se muestrea según estas probabilidades

**Efecto de temperature:**
Los logits se dividen por temperature ANTES del softmax:
```python
adjusted_logits = logits / temperature
probabilities = softmax(adjusted_logits)
```

**Ejemplo numérico:**
```
Logits originales: ["es": 3.0, "fue": 2.0, "era": 1.0]

Temperature 0.5 (conservador):
  Logits ajustados: [6.0, 4.0, 2.0]
  Probabilidades: [0.84, 0.14, 0.02]
  → "es" domina fuertemente

Temperature 1.0 (neutral):
  Logits ajustados: [3.0, 2.0, 1.0]
  Probabilidades: [0.67, 0.24, 0.09]
  → "es" sigue siendo más probable pero otros tienen más chance

Temperature 2.0 (creativo):
  Logits ajustados: [1.5, 1.0, 0.5]
  Probabilidades: [0.48, 0.32, 0.20]
  → Distribución más uniforme, más variedad
```

**Por qué genera texto más creativo:**
- Temperature alta → Probabilidades más uniformes
- Tokens menos probables ganan chance de ser seleccionados
- Resulta en combinaciones de palabras menos predecibles
- Mayor exploración del espacio de posibilidades

**Trade-off:**
- Temperature muy baja (0.1): Casi determinístico, repetitivo
- Temperature balanceada (0.7-0.9): Creativo pero coherente
- Temperature muy alta (> 1.5): Muy aleatorio, puede ser incoherente
</details>

---

**4. ¿Cuál es el límite de contexto de Phi-3 y qué implicaciones tiene para conversaciones largas?**

<details>
<summary>Ver respuesta</summary>

**Límite de Phi-3-mini-4k-instruct:**
- 4096 tokens totales (aproximadamente 3000 palabras en español)
- Incluye: prompt + historial de conversación + nueva generación

**Implicaciones:**

**1. Conversaciones largas:**
Si la conversación acumula muchos turnos, puede exceder el límite:
```
Turno 1: 500 tokens
Turno 2: 500 tokens
Turno 3: 600 tokens
...
Turno 8: Supera 4096 → Error
```

**2. Documentos largos:**
No podés analizar documentos que excedan ~3000 palabras en un solo prompt.

**Estrategias de mitigación:**

**A) Sliding Window (ventana deslizante):**
```python
# Mantener solo últimos N turnos
MAX_TURNOS = 5
if len(conversacion) > MAX_TURNOS * 2:  # *2 porque cada turno tiene user + assistant
    conversacion = conversacion[-(MAX_TURNOS * 2):]
```

**B) Summarization (resumir conversación antigua):**
```python
# Cada 10 turnos, resume los primeros 5
resumen = model.generate("Resume esta conversación: ...")
conversacion_nueva = [{"role": "system", "content": resumen}] + conversacion[-10:]
```

**C) Retrieval (recuperar contexto relevante):**
- Guardar toda la conversación en base de datos
- Recuperar solo los turnos relevantes a la consulta actual
- Usar embeddings para búsqueda semántica

**D) Chunking (para documentos largos):**
```python
# Dividir documento en chunks de 3000 tokens
# Procesar cada chunk
# Combinar resultados
```

**Modelos con mayor contexto:**
- Phi-3-medium-128k: 128,000 tokens
- GPT-4-turbo: 128,000 tokens
- Claude 2: 100,000 tokens

Pero estos son más grandes/caros/lentos.
</details>

---

**5. ¿Qué es "few-shot learning" y cómo difiere de entrenar un modelo desde cero?**

<details>
<summary>Ver respuesta</summary>

**Few-Shot Learning:**
Proporcionar pocos ejemplos (típicamente 1-10) dentro del prompt para que el modelo infiera el patrón:

```python
prompt = """
Traducí estas frases:

EN: Hello → ES: Hola
EN: Thank you → ES: Gracias
EN: Good morning → ES: Buenos días

EN: How are you? → ES:
"""
```

El modelo infiere que debe traducir sin haber sido entrenado específicamente para traducción.

**Entrenar desde cero:**
- Requiere miles/millones de ejemplos etiquetados
- Ajusta los pesos del modelo mediante backpropagation
- Tiempo: Horas/días de entrenamiento
- Costo: Alto (GPUs, electricidad, datos etiquetados)
- Resultado: Modelo especializado en la tarea

**Comparación:**

| Aspecto | Few-Shot | Entrenamiento |
|---------|----------|---------------|
| Ejemplos necesarios | 1-10 | Miles/millones |
| Tiempo | Instantáneo | Horas/días |
| Costo | Bajo | Alto |
| Modificación del modelo | No (solo prompt) | Sí (pesos) |
| Precisión | Moderada | Alta (si bien entrenado) |
| Flexibilidad | Alta (cambiar ejemplos) | Baja (reentrenar) |

**Cuándo usar cada uno:**

**Few-Shot:**
- Prototipado rápido
- Tareas que cambian frecuentemente
- Pocos datos disponibles
- Precisión moderada es suficiente

**Entrenamiento:**
- Producción con alto volumen
- Necesitás máxima precisión
- Tenés muchos datos etiquetados
- La tarea es fija

**Enfoque híbrido (Fine-Tuning):**
Ajustar un modelo preentrenado con pocos cientos de ejemplos de tu tarea específica:
- Más rápido/barato que entrenar desde cero
- Más preciso que few-shot
- Balance óptimo para muchos casos
</details>

---

### Preguntas de Aplicación

**6. Diseñá un prompt para que Phi-3 actúe como asistente de código Python, explicando errores y sugiriendo correcciones.**

<details>
<summary>Ver respuesta</summary>

```python
mensaje_asistente_codigo = [
    {
        "role": "user",
        "content": """Actuá como un asistente experto en Python. Tu tarea es:
1. Identificar todos los errores en el código
2. Explicar por qué cada error es problemático
3. Proporcionar la corrección
4. Sugerir mejores prácticas si es relevante

Código con errores:
```python
def calcular_promedio(numeros):
    total = 0
    for num in numeros:
        total =+ num
    promedio = total / len(numeros)
    return promedio

lista = [10, 20, 30]
resultado = calcular_promedio(lista)
print("El promedio es: " + resultado)
```

Analizá el código y proporcioná tu respuesta estructurada."""
    }
]

# Configuración para respuesta técnica precisa
gen_tecnico = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=400,
    do_sample=False  # Determinístico para consistencia técnica
)

output = gen_tecnico(mensaje_asistente_codigo)
print(output[0]["generated_text"])
```

**Elementos clave del prompt:**
- Rol específico ("experto en Python")
- Tareas numeradas (estructura clara)
- Código en bloques de código (formato)
- Solicita respuesta estructurada

**Mejoras posibles:**
- Agregar ejemplos de análisis previos (few-shot)
- Especificar nivel de detalle deseado
- Pedir que sugiera tests para el código corregido
</details>

---

**7. Implementá un sistema que genere múltiples variaciones de un slogan publicitario y las evalúe según criterios específicos.**

<details>
<summary>Ver respuesta</summary>

```python
def generar_y_evaluar_slogans(producto, num_variaciones=5):
    """
    Genera múltiples slogans y los evalúa.
    """
    # 1. Generar variaciones
    gen_creativo = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        return_full_text=False,
        max_new_tokens=100,
        do_sample=True,
        temperature=1.0,  # Alta creatividad
        top_p=0.95
    )
    
    prompt_generacion = [
        {
            "role": "user",
            "content": f"""Generá {num_variaciones} slogans publicitarios creativos y memorables
para: {producto}

Características deseadas:
- Cortos (máximo 8 palabras)
- Memorables y pegajosos
- Positivos y motivadores
- Que reflejen el valor del producto

Formato: Numerá cada slogan."""
        }
    ]
    
    output_generacion = gen_creativo(prompt_generacion)
    slogans_generados = output_generacion[0]["generated_text"]
    
    print("SLOGANS GENERADOS:")
    print("="*80)
    print(slogans_generados)
    print("\n")
    
    # 2. Evaluar slogans
    gen_evaluacion = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        return_full_text=False,
        max_new_tokens=400,
        do_sample=False  # Evaluación consistente
    )
    
    prompt_evaluacion = [
        {
            "role": "user",
            "content": f"""Evaluá cada uno de estos slogans según estos criterios (puntaje 1-10):
- Memorabilidad: ¿Qué tan fácil es recordarlo?
- Impacto: ¿Qué tan poderoso es el mensaje?
- Claridad: ¿Comunica claramente el valor?
- Originalidad: ¿Qué tan único es?

Slogans:
{slogans_generados}

Para cada slogan, proporcioná:
1. Puntajes por criterio
2. Puntaje total (promedio)
3. Análisis breve
4. Sugerencias de mejora

Al final, indicá cuál es el mejor slogan y por qué."""
        }
    ]
    
    output_evaluacion = gen_evaluacion(prompt_evaluacion)
    
    print("EVALUACIÓN:")
    print("="*80)
    print(output_evaluacion[0]["generated_text"])

# Ejemplo de uso
generar_y_evaluar_slogans("Una app de meditación y mindfulness para reducir el estrés")
```

**Técnicas usadas:**
- Diferentes configuraciones para generación vs evaluación
- Temperature alta para creatividad en generación
- Determinístico para evaluación consistente
- Criterios específicos y cuantificables
- Pipeline de dos etapas
</details>

---

**8. El modelo genera respuestas demasiado largas incluso con max_new_tokens bajo. ¿Qué estrategias de prompt podrías usar para obtener respuestas más concisas?**

<details>
<summary>Ver respuesta</summary>

Estrategias para forzar concisión:

**1. Especificar restricciones numéricas:**
```python
"Respondé en EXACTAMENTE 3 oraciones."
"Tu respuesta debe tener máximo 50 palabras."
"Resumí en un solo párrafo de 4-5 líneas."
```

**2. Pedir formatos específicos:**
```python
"Respondé en formato de lista con 5 puntos."
"Proporcioná solo la definición, sin ejemplos."
"Dame solo la respuesta directa, sin introducción ni conclusión."
```

**3. Usar instrucciones imperativas:**
```python
"Sé extremadamente conciso."
"Solo lo esencial."
"Versión ultra-breve."
```

**4. Few-shot con ejemplos concisos:**
```python
mensaje = [
    {
        "role": "user",
        "content": """Respondé preguntas de forma concisa. Ejemplos:

P: ¿Qué es la fotosíntesis?
R: Proceso por el cual las plantas convierten luz solar en energía química.

P: ¿Cuándo fue la Revolución Francesa?
R: 1789-1799.

Ahora respondé:
P: ¿Qué es un modelo de lenguaje?
R:"""
    }
]
```

**5. Instrucciones negativas:**
```python
"No incluyas ejemplos ni explicaciones adicionales."
"No des contexto histórico, solo la información solicitada."
```

**6. Formato de telegrama:**
```python
"Respondé en estilo telegrama: solo información clave, sin palabras innecesarias."
```

**7. Solicitar tabla o estructura:**
```python
"Presentá la información en una tabla con 3 columnas."
# Las tablas naturalmente limitan la verbosidad
```

**Ejemplo completo:**
```python
mensaje_conciso = [
    {
        "role": "user",
        "content": """Explicá qué es la inteligencia artificial.

RESTRICCIONES:
- Máximo 2 oraciones
- Solo la definición, sin historia ni ejemplos
- Lenguaje simple"""
    }
]
```

**Nota:** Si el modelo aún genera texto largo, puede ser que max_new_tokens sea muy alto. Combinar restricciones en el prompt con max_new_tokens=50-100.
</details>

---

**9. Diseñá un sistema conversacional que mantenga el contexto pero gestione eficientemente el límite de tokens.**

<details>
<summary>Ver respuesta</summary>

```python
class ChatbotConGestionContexto:
    def __init__(self, max_history_tokens=3000):
        self.conversacion = []
        self.max_history_tokens = max_history_tokens
        self.generator = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            return_full_text=False,
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7
        )
    
    def contar_tokens(self, texto):
        """Cuenta tokens en un texto."""
        return len(tokenizer.encode(texto))
    
    def contar_tokens_conversacion(self):
        """Cuenta tokens totales en la conversación actual."""
        total = 0
        for msg in self.conversacion:
            total += self.contar_tokens(msg["content"])
        return total
    
    def gestionar_contexto(self):
        """Mantiene la conversación dentro del límite de tokens."""
        while self.contar_tokens_conversacion() > self.max_history_tokens:
            if len(self.conversacion) > 2:
                # Eliminar el par user-assistant más antiguo (mantener al menos 1 turno)
                self.conversacion.pop(0)  # Remove user
                if len(self.conversacion) > 0 and self.conversacion[0]["role"] == "assistant":
                    self.conversacion.pop(0)  # Remove assistant
            else:
                break
    
    def chat(self, mensaje_usuario):
        """Procesa un mensaje del usuario y genera respuesta."""
        # Agregar mensaje del usuario
        self.conversacion.append({
            "role": "user",
            "content": mensaje_usuario
        })
        
        # Gestionar contexto
        self.gestionar_contexto()
        
        # Generar respuesta
        output = self.generator(self.conversacion)
        respuesta = output[0]["generated_text"]
        
        # Agregar respuesta del asistente
        self.conversacion.append({
            "role": "assistant",
            "content": respuesta
        })
        
        return respuesta
    
    def mostrar_estadisticas(self):
        """Muestra información sobre el estado actual."""
        tokens_actuales = self.contar_tokens_conversacion()
        print(f"Tokens en conversación: {tokens_actuales} / {self.max_history_tokens}")
        print(f"Turnos en conversación: {len(self.conversacion) // 2}")
    
    def resetear(self):
        """Reinicia la conversación."""
        self.conversacion = []

# Uso
chatbot = ChatbotConGestionContexto(max_history_tokens=3000)

# Conversación de ejemplo
print("=" * 80)
respuesta1 = chatbot.chat("Hola, ¿podés explicarme qué es la programación orientada a objetos?")
print(f"Usuario: Hola, ¿podés explicarme qué es la programación orientada a objetos?")
print(f"Asistente: {respuesta1}")
chatbot.mostrar_estadisticas()

print("\n" + "=" * 80 + "\n")
respuesta2 = chatbot.chat("¿Y cuáles son sus principales ventajas?")
print(f"Usuario: ¿Y cuáles son sus principales ventajas?")
print(f"Asistente: {respuesta2}")
chatbot.mostrar_estadisticas()

print("\n" + "=" * 80 + "\n")
respuesta3 = chatbot.chat("Dame un ejemplo práctico en Python.")
print(f"Usuario: Dame un ejemplo práctico en Python.")
print(f"Asistente: {respuesta3}")
chatbot.mostrar_estadisticas()
```

**Mejoras avanzadas:**

1. **Resumen de contexto antiguo:**
```python
def resumir_contexto_antiguo(self):
    if len(self.conversacion) > 10:
        # Resumir primeros 6 mensajes
        contexto_antiguo = self.conversacion[:6]
        texto_resumir = "\n".join([f"{m['role']}: {m['content']}" for m in contexto_antiguo])
        
        resumen = self.generator([{
            "role": "user",
            "content": f"Resume brevemente esta conversación:\n{texto_resumir}"
        }])[0]["generated_text"]
        
        # Reemplazar con resumen
        self.conversacion = [
            {"role": "system", "content": f"Contexto previo: {resumen}"}
        ] + self.conversacion[6:]
```

2. **Priorizar mensajes recientes:**
Mantener siempre los últimos N turnos completos, eliminar los más antiguos.

3. **Detección de cambio de tema:**
Si el usuario cambia de tema, opcionalmente eliminar contexto no relevante.
</details>

---

**10. Implementá un sistema que genere código Python funcional a partir de descripciones en lenguaje natural y lo valide ejecutándolo.**

<details>
<summary>Ver respuesta</summary>

```python
import re
import sys
from io import StringIO

def generar_codigo_desde_descripcion(descripcion):
    """
    Genera código Python desde descripción natural.
    """
    gen_codigo = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        return_full_text=False,
        max_new_tokens=500,
        do_sample=False  # Código debe ser determinístico
    )
    
    prompt = [
        {
            "role": "user",
            "content": f"""Generá código Python funcional y completo para esta tarea:

{descripcion}

REQUISITOS:
- Código completo y ejecutable
- Incluir comentarios explicativos
- Manejar casos edge
- Incluir docstring
- Proporcionar un ejemplo de uso al final

Devolvé SOLO el código Python, sin texto adicional antes o después."""
        }
    ]
    
    output = gen_codigo(prompt)
    codigo_generado = output[0]["generated_text"]
    
    # Extraer solo el código (eliminar markdown si existe)
    codigo_limpio = extraer_codigo_python(codigo_generado)
    
    return codigo_limpio

def extraer_codigo_python(texto):
    """Extrae código Python de texto que puede incluir markdown."""
    # Buscar bloques de código markdown
    match = re.search(r'```python\n(.+?)```', texto, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # Si no hay markdown, devolver todo
    return texto.strip()

def validar_codigo(codigo):
    """
    Valida que el código sea sintácticamente correcto.
    """
    try:
        compile(codigo, '<string>', 'exec')
        return True, "Sintaxis válida"
    except SyntaxError as e:
        return False, f"Error de sintaxis: {e}"

def ejecutar_codigo_seguro(codigo, timeout=5):
    """
    Ejecuta código en un entorno controlado y captura la salida.
    
    ADVERTENCIA: En producción, usar sandbox real (Docker, etc.)
    Este es solo para demostración educativa.
    """
    # Capturar stdout
    old_stdout = sys.stdout
    sys.stdout = buffer = StringIO()
    
    try:
        # Ejecutar código
        exec(codigo, {"__builtins__": __builtins__})
        output = buffer.getvalue()
        sys.stdout = old_stdout
        return True, output, None
    except Exception as e:
        sys.stdout = old_stdout
        return False, None, str(e)

def pipeline_completo(descripcion):
    """
    Pipeline completo: generación, validación y ejecución.
    """
    print("="*80)
    print("DESCRIPCIÓN:")
    print(descripcion)
    print("\n" + "="*80)
    
    # 1. Generar código
    print("\nGenerando código...")
    codigo = generar_codigo_desde_descripcion(descripcion)
    
    print("\nCÓDIGO GENERADO:")
    print("-"*80)
    print(codigo)
    print("-"*80)
    
    # 2. Validar sintaxis
    print("\nValidando sintaxis...")
    valido, mensaje_validacion = validar_codigo(codigo)
    print(f"Validación: {mensaje_validacion}")
    
    if not valido:
        print("\n❌ El código no es sintácticamente válido.")
        return
    
    # 3. Ejecutar
    print("\nEjecutando código...")
    exito, output, error = ejecutar_codigo_seguro(codigo)
    
    if exito:
        print("\n✓ Ejecución exitosa")
        if output:
            print("\nSALIDA:")
            print(output)
    else:
        print(f"\n❌ Error en ejecución: {error}")

# Ejemplos de uso
pipeline_completo("""
Escribí una función que reciba una lista de números y devuelva:
1. El promedio
2. El valor máximo
3. El valor mínimo

La función debe manejar listas vacías devolviendo None para cada métrica.
Incluí un ejemplo probando con la lista [10, 25, 8, 42, 15].
""")
```

**Mejoras para producción:**

1. **Sandbox real:** Usar Docker o máquinas virtuales para ejecutar código de manera aislada
2. **Límites de recursos:** Timeout, memoria máxima, no permitir I/O de archivos
3. **Validación semántica:** Verificar que el código hace lo que se pidió usando tests automáticos
4. **Iteración:** Si falla, pedirle al modelo que corrija el error
5. **Tests unitarios:** Generar también tests para el código
</details>

---

### Desafíos Adicionales

**Desafío 1**: Implementá un sistema que genere descripciones de productos para e-commerce optimizadas para SEO, incluyendo título, descripción, bullet points y palabras clave.

**Desafío 2**: Creá un asistente que ayude a redactar currículums vitae, preguntando información al usuario y generando secciones profesionales.

**Desafío 3**: Diseñá un sistema de generación de contenido educativo que, dada un tema, genere: explicación teórica, ejemplos prácticos, ejercicios y soluciones.

---

## Referencias y Recursos Adicionales

- [Phi-3 Technical Report](https://arxiv.org/abs/2404.14219)
- [Phi-3 Model Card](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [Hugging Face Text Generation](https://huggingface.co/docs/transformers/main_classes/text_generation)
- [Few-Shot Learning for NLP](https://arxiv.org/abs/2005.14165)
- [Chain-of-Thought Prompting](https://arxiv.org/abs/2201.11903)